# Making a Conversational Form

### Setting up Conversation Filtering



In [1]:
from langchain_openai import AzureChatOpenAI
from dotenv import load_dotenv
import os
from typing import Optional
from pydantic import BaseModel, Field

load_dotenv()

llm = AzureChatOpenAI(
    azure_endpoint=os.getenv("AISTUDIO_AZURE_OPENAI_ENDPOINT"),
    azure_deployment=os.getenv("AI_STUDIO_AZURE_OPENAI_GPT4o_DEPLOYMENT"),
    openai_api_version="2024-02-01",
    api_key=os.getenv("AISTUDIO_AZURE_OPENAI_KEY")
)

In [2]:
class PersonalDetails(BaseModel):
    first_name: str = Field(
        ...,
        description="This is the first name of the user.",
    )
    last_name: str = Field(
        ...,
        description="This is the last name or surname of the user.",
    )
    full_name: str = Field(
        ...,
        description="Is the full name of the user ",
    )
    phone_nr: str = Field(
        ...,
        description="Is the phone number of the user ",
    )
    city: str = Field(
        ...,
        description="The name of the city where someone lives",
    )
    email: str = Field(
        ...,
        description="an email address that the person associates as theirs",
    )
    language: str = Field(
        ..., enum=["spanish", "english", "french", "german", "italian"]
    )
structured_llm = llm.with_structured_output(PersonalDetails)


In [3]:
structured_llm.invoke("Hi my name is David Jones and I live in Melbourne Australia.")

PersonalDetails(first_name='David', last_name='Jones', full_name='David Jones', phone_nr='', city='Melbourne', email='', language='english')

In [4]:
res = structured_llm.invoke("Hi my name is Chatree Kongsuwan  and I live in Bangkok. you can contact me at chatree@gmail.com")
print(res)
print(res.email)

first_name='Chatree' last_name='Kongsuwan' full_name='Chatree Kongsuwan' phone_nr='' city='Bangkok' email='chatree@gmail.com' language='english'
chatree@gmail.com


## Filling gradually

In [5]:
user_personal_details = PersonalDetails(first_name="",
                                last_name="",
                                full_name="",
                                phone_nr="",
                                city="",
                                email="",
                                language="")

In [6]:
user_personal_details

PersonalDetails(first_name='', last_name='', full_name='', phone_nr='', city='', email='', language='')

In [7]:
def check_what_is_empty(user_personal_details):
    ask_for = []
    # Check if fields are empty
    dictionary = user_personal_details.model_dump(exclude_unset=True)
    # check if the field is empty
    for field, value in dictionary.items():
        if value in [None, "", 0]:  # You can add other 'empty' conditions as per your requirements
            print(f"Please fill the value for:  '{field}'.")
            ask_for.append(f'{field}')
    return ask_for

## checking the response and adding it
def add_non_empty_details(current_details: PersonalDetails, new_details: PersonalDetails):
    new_dict = new_details.model_dump(exclude_unset=True)
    non_empty_details = {k: v for k, v in new_dict.items() if v not in [None, ""]}
    updated_details = current_details.model_copy(update=non_empty_details)
    return updated_details

In [8]:
ask_for = check_what_is_empty(user_personal_details)

Please fill the value for:  'first_name'.
Please fill the value for:  'last_name'.
Please fill the value for:  'full_name'.
Please fill the value for:  'phone_nr'.
Please fill the value for:  'city'.
Please fill the value for:  'email'.
Please fill the value for:  'language'.


In [9]:
res = structured_llm.invoke("Hi my name is John Smith and I live in London UK. you can contact me at john123@gmail.com")
print(res)

user_123_personal_details = add_non_empty_details(user_personal_details,res)
user_123_personal_details

first_name='John' last_name='Smith' full_name='John Smith' phone_nr='' city='London' email='john123@gmail.com' language='english'


PersonalDetails(first_name='John', last_name='Smith', full_name='John Smith', phone_nr='', city='London', email='john123@gmail.com', language='english')

In [10]:
# check if the fields are empty
ask_for = check_what_is_empty(user_personal_details)

Please fill the value for:  'first_name'.
Please fill the value for:  'last_name'.
Please fill the value for:  'full_name'.
Please fill the value for:  'phone_nr'.
Please fill the value for:  'city'.
Please fill the value for:  'email'.
Please fill the value for:  'language'.


In [11]:
#adding more information based on the response
res = structured_llm.invoke("My phone number is 123456789")

user_personal_details = add_non_empty_details(user_personal_details,res)
user_personal_details

PersonalDetails(first_name='John', last_name='Doe', full_name='John Doe', phone_nr='123456789', city='New York', email='john.doe@example.com', language='english')

## Putting it together with a LLMChain as well


In [26]:
from langchain.chains import TransformChain, LLMChain, SimpleSequentialChain
from langchain.prompts import PromptTemplate, ChatPromptTemplate

In [27]:
user_123_personal_details = PersonalDetails(first_name="",
                                last_name="",
                                full_name="",
                                phone_nr="",
                                city="",
                                email="",
                                language="")

In [28]:
user_123_personal_details

PersonalDetails(first_name='', last_name='', full_name='', phone_nr='', city='', email='', language='')

In [31]:
def ask_for_info(required_fields):
    # prompt template 1
    first_prompt = ChatPromptTemplate.from_template(
        "Below is are some things to ask the user for in a conversational way. You should only ask one question at a time even if you don't get all the info \
        don't ask as a list! Don't greet the user! Don't say Hi. Explain you need to get some info. If the required_fields list is empty then thank them and ask how you can help them \n\n \
        ### required_fields list: {required_fields}"
    )
    # info_gathering_chain
    info_gathering_chain = first_prompt | structured_llm
    ai_chat = info_gathering_chain.invoke({"required_fields": required_fields})
    return ai_chat


In [36]:
text_input ="ok My name is Sam"
user_details = ask_for_info(user_123_personal_details)
print(user_details)

first_name='' last_name='' full_name='' phone_nr='' city='' email='' language='english'
